In [1]:
#######################Tensorflow 코드 시작부분
import librosa
import pyaudio #마이크를 사용하기 위한 라이브러리
import wave
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
import os

##### 변수 설정 부분 #####
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100 #비트레이트 설정
CHUNK = int(RATE / 10) # 버퍼 사이즈 1초당 44100비트레이트 이므로 100ms단위
RECORD_SECONDS = 5 #녹음할 시간 설정
WAVE_OUTPUT_FILENAME = "output.wav"
DATA_PATH = "./data/"
X_train = []#train_data 저장할 공간
X_test = []
Y_train = []
Y_test = []
tf_classes = 0

def load_wave_generator(path):

    batch_waves = []
    labels = []
    X_data = []
    Y_label = []
    global X_train, X_test, Y_train, Y_test, tf_classes

    folders = os.listdir(path)

    for folder in folders:
        if not os.path.isdir(path):continue #폴더가 아니면 continue
        files = os.listdir(path+"/"+folder)
        print("Foldername :",folder,"-",len(files),"files")
        #폴더 이름과 그 폴더에 속하는 파일 갯수 출력
        for wav in files:
            if not wav.endswith(".wav"):continue
            else:
                #print("Filename :",wav)#.wav 파일이 아니면 continue
                y, sr = librosa.load(path+"/"+folder+"/"+wav)
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

                X_data.extend(mfcc)
               # print(len(mfcc))

                label = [0 for i in range(len(folders))]
                label[tf_classes] = 1

                for i in range(len(mfcc)):
                    Y_label.append(label)
        tf_classes = tf_classes+1
    #end loop
    print("X_data :",np.shape(X_data))
    print("Y_label :",np.shape(Y_label))
    X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_data), np.array(Y_label))

    xy = (X_train, X_test, Y_train, Y_test)
    np.save("./data.npy",xy)

print("Start FIle Reading")
load_wave_generator(DATA_PATH)

print(tf_classes," classes!!")
print("X_train :",np.shape(X_train))
print("Y_train :",np.shape(Y_train))
print("X_test :",np.shape(X_test))
print("Y_test :",np.shape(Y_test))

Foldername : 0 - 20 files
Foldername : 1 - 20 files
Foldername : 2 - 20 files
Foldername : 3 - 20 files
Foldername : 4 - 20 files
X_data : (50200, 13)
Y_label : (50200, 5)
5  classes!!
X_train : (37650, 13)
Y_train : (37650, 5)
X_test : (12550, 13)
Y_test : (12550, 5)


C:\Users\alexr\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [2]:
print(X_train[0])
print(Y_train[0])

[-274.48743     109.914566    -23.722542     11.754871      3.0201812
    1.0277314    19.445402     22.75625       6.366988      1.3752742
    0.28390282  -14.3806305   -10.07159   ]
[1 0 0 0 0]


In [38]:
X_train, X_test, Y_train, Y_test = np.load("./data.npy", allow_pickle=True)
X_train = X_train.astype("float")
X_test = X_test.astype("float")


class MyModel(tf.keras.Model):             #모델 클래스 선언
    
    def __init__(self):
        super(MyModel,self).__init__()
        self.dense1 = layers.Dense(256, activation='relu')
        self.dense2 = layers.Dense(128, activation='relu')
        self.dense3 = layers.Dense(64, activation='relu')
        self.dense4 = layers.Dense(32, activation='relu')
        self.dense5 = layers.Dense(64, activation='relu')
        self.dense6 = layers.Dense(32, activation='relu')
        self.dense7 = layers.Dense(16, activation='relu')
        self.dense8 = layers.Dense(5, activation='softmax')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        x = self.dense7(x)
        return self.dense8(x)

batch_size = 32
model = MyModel()
model.compile(optimizer='adam',
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=50, batch_size=batch_size)


print('Learning Finished!')


Epoch 1/50
1177/1177 [==============================] - 1s 752us/step - loss: 0.9220 - accuracy: 0.6360
Epoch 2/50
1177/1177 [==============================] - 1s 759us/step - loss: 0.6611 - accuracy: 0.7477
Epoch 3/50
1177/1177 [==============================] - 1s 755us/step - loss: 0.5823 - accuracy: 0.7806
Epoch 4/50
1177/1177 [==============================] - 1s 756us/step - loss: 0.5328 - accuracy: 0.8018
Epoch 5/50
1177/1177 [==============================] - 1s 758us/step - loss: 0.4931 - accuracy: 0.8177
Epoch 6/50
1177/1177 [==============================] - 1s 768us/step - loss: 0.4702 - accuracy: 0.8280
Epoch 7/50
1177/1177 [==============================] - 1s 755us/step - loss: 0.4481 - accuracy: 0.8352
Epoch 8/50
1177/1177 [==============================] - 1s 754us/step - loss: 0.4338 - accuracy: 0.8426
Epoch 9/50
1177/1177 [==============================] - 1s 753us/step - loss: 0.4176 - accuracy: 0.8494
Epoch 10/50
1177/1177 [==============================] - 1s 778u

In [ ]:
saver = model.save("MFCC_DNN")    #모델 저장

In [33]:
y, sr = librosa.load("./test_이재은.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))

pridict = model.predict(X_test)
rst = [0,0,0,0,0]
for i in range(len(pridict)):
    rst[np.argmax(pridict[i])] = rst[np.argmax(pridict[i])] +1
print(rst)
accuracy = (rst[2]/len(pridict))*100
print("accuracy is ", accuracy)

(502, 13)
(502, 5)
[1, 0, 500, 0, 1]
accuracy is  99.60159362549801


In [34]:
y, sr = librosa.load("./test_유인나1.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))

pridict = model.predict(X_test)
rst = [0,0,0,0,0]
for i in range(len(pridict)):
    rst[np.argmax(pridict[i])] = rst[np.argmax(pridict[i])] +1
print(rst)
accuracy = (rst[0]/len(pridict))*100
print("accuracy is ", accuracy)

(502, 13)
(502, 5)
[443, 17, 23, 4, 15]
accuracy is  88.24701195219123


In [35]:
y, sr = librosa.load("./test_유인나2.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))

pridict = model.predict(X_test)
rst = [0,0,0,0,0]
for i in range(len(pridict)):
    rst[np.argmax(pridict[i])] = rst[np.argmax(pridict[i])] +1
print(rst)
accuracy = (rst[0]/len(pridict))*100
print("accuracy is ", accuracy)

(502, 13)
(502, 5)
[469, 11, 2, 18, 2]
accuracy is  93.42629482071713


In [36]:
y, sr = librosa.load("./test_배철수.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))

pridict = model.predict(X_test)
rst = [0,0,0,0,0]
for i in range(len(pridict)):
    rst[np.argmax(pridict[i])] = rst[np.argmax(pridict[i])] +1
print(rst)
accuracy = (rst[1]/len(pridict))*100
print("accuracy is ", accuracy)

(502, 13)
(502, 5)
[9, 425, 29, 27, 12]
accuracy is  84.66135458167331


In [37]:
y, sr = librosa.load("./test_문재인대통령.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))

pridict = model.predict(X_test)
rst = [0,0,0,0,0]
for i in range(len(pridict)):
    rst[np.argmax(pridict[i])] = rst[np.argmax(pridict[i])] +1
print(rst)
accuracy = (rst[4]/len(pridict))*100
print("accuracy is ", accuracy)

(502, 13)
(502, 5)
[27, 64, 46, 48, 317]
accuracy is  63.147410358565736
